## 코드 추상화: 클래스와 객체 2부

안내: [python-textbook.readthedocs.io의 Classes](https://python-textbok.readthedocs.io/en/1.0/Classes.html#) 내용을 요약 및 수정한 내용입니다.

## 메서드 종류

클래스에서 선언된 메서드는 세 종류로 나뉜다.

1. 인스턴스 메서드(instance method)
    * 첫째 매개변수로 `self` 사용 (관례)
    * 클래스의 인스턴스가 생성된 후에 인스턴스와 함께 사용.
1. 클래스 메서드(class method)
    * 첫째 매개변수로 `cls` 사용 (관례)
    * 클래스 내의 모든 속성과 메서드를 `cls` 지정자와 함께 사용 가능
        단, 인스턴스 속성 사용 불가.
    * 인스턴스 없이 클래스 이름과 함께 사용
    * 해당 클래스의 모든 인스턴스에서도 사용 가능
1. 정적 메서드(static method)
    * 첫째 매개변수에 대한 의무사항 없음.
    * 클래스 내의 모든 속성과 메서드를 클래스 이름을 지정자와 함께 사용해야 함.
        단, 인스턴스 속성 사용 불가.
    * 인스턴스 없이 클래스 이름과 함께 사용
    * 해당 클래스의 모든 인스턴스에서도 사용 가능
    
`self`를 첫째 매개변수로 사용하는 인스턴스 메서드는 특별한 장식자가 필요 없다.
예를 들어, 
[코드 추상화: 클래스와 객체 1부](https://github.com/liganega/ProgInPython/blob/master/notebooks/PiPy08-ClassesAndInstances_Part1.ipynb)에서 
살펴본 모든 메서드는 인스턴스 메서드이다. 
반면에 클래스 메서드와 정적 메서드는 항상 장식자와 함께 선언된다.

### 클래스 장식자

**장식자**(decorator)는 다른 함수의 기능에 다른 기능을 추가할 때 사용되는 함수이다.
즉, 장식자는 함수를 인자로 받아 그 함수가 하는 일에 더해 다른 기능도
수행하는 함수를 리턴값으로 내준다.
이런 장식자를 함수로 정의할 수 있는 이유는 함수가 제1종 객체이기 때문이다.
즉, 다른 함수의 인자 또는 리턴값으로 사용될 수 있다.

**참고:** 장식자를 기존의 함수에 유용한 추가기능을 제공하여
포장한다는 의미에서 **래퍼**(wrapper)의 일종으로
간주한다. 

파이썬에서 기본으로 제공하는 장식자가 매우 다양하며, 사용자가 직접 장식자를 정의할 수도 있다.
여기서는 클래스에서 선언된 메서드의 종류를 구분하기 위해 사용되는 
두 개의 장식자
`@classmethod`와 `@staticmethod`를 소개한다. 

### 클래스 메서드 장식자: `@classmethod`

클래스의 인스턴스를 생성하지 않아도 클래스 이름과 함께 사용할 수 있는 
메서드를 **클래스 메서드**(class method)라 부르며,
`@classmethod`라는 장식자와 함께 선언된다. 

```python
@classmethod
def 함수이름(cls,인자1, ..., 인자k):
    본문
```

클래스 메서드 역시 첫째 인자로 해당 클래스를 받을 준비를 하는 매개변수를 반드시 사용해야 한다.
하지만 `self` 대신에 클래스(class) 자체를 가리킨다는 의미로 `cls`를 관례적으로 사용한다. 
클래스 메서드를 호출하면 `cls`에 해당 클래스 이름이 자동으로 삽입된다.
따라서 클래스 메서드를 호출할 때 첫째 인자는 생략한다. 

클래스 메서드를 선언할 때 사용되는 `cls` 매개변수는 클래스 자신을 가리키는 
지정자 역할을 수행한다. 
따라서 클래스 메서드 내부에서는 인스턴스 속성과 인스턴스 메서드를 활용하지 못한다. 

#### 클래스 메서드 활용법

클래스 메서드를 사용하는 이유는 크게 두 가지이다. 

첫째, 상수(constant) 역할을 수행하는 값이나 클래스 속성을 직접 활용하고자 할 때 유용하다.
이는 상수와 클래스 속성을 활용하기 위해 특정 객체가 필요하지 않기 때문이다. 
또한 경우에 따라 연관된 상수나 함수들을 하나로 클래스로 묶어서 활용할 수도 있다.
이런 경우 굳이 객체를 생성할 필요가 없다. 

In [1]:
class Title:
    TITLES = ('Dr', 'Mr', 'Mrs', 'Ms')

    @classmethod
    def allowed_titles_starting_with(cls, startswith):
        # startwith로 시작하는 타이틀 찾기
        return [t for t in cls.TITLES if t.startswith(startswith)]

    @classmethod
    def allowed_titles_ending_with(cls, endswith):
        # endswith로 끝나는 타이틀 찾기
        return [t for t in cls.TITLES if t.endswith(endswith)]


print(Title.allowed_titles_starting_with("M"))
print(Title.allowed_titles_ending_with("s"))

['Mr', 'Mrs', 'Ms']
['Mrs', 'Ms']


#### PythonTutor 활용 1

위 코드를 
[PythonTutor: 클래스 장식자 예제 1](http://pythontutor.com/visualize.html#code=class%20Title%3A%0A%20%20%20%20TITLES%20%3D%20%28'Dr',%20'Mr',%20'Mrs',%20'Ms'%29%0A%0A%20%20%20%20%40classmethod%0A%20%20%20%20def%20allowed_titles_starting_with%28cls,%20startswith%29%3A%0A%20%20%20%20%20%20%20%20%23%20startwith%EB%A1%9C%20%EC%8B%9C%EC%9E%91%ED%95%98%EB%8A%94%20%ED%83%80%EC%9D%B4%ED%8B%80%20%EC%B0%BE%EA%B8%B0%0A%20%20%20%20%20%20%20%20return%20%5Bt%20for%20t%20in%20cls.TITLES%20if%20t.startswith%28startswith%29%5D%0A%0A%20%20%20%20%40classmethod%0A%20%20%20%20def%20allowed_titles_ending_with%28cls,%20endswith%29%3A%0A%20%20%20%20%20%20%20%20%23%20endswith%EB%A1%9C%20%EB%81%9D%EB%82%98%EB%8A%94%20%ED%83%80%EC%9D%B4%ED%8B%80%20%EC%B0%BE%EA%B8%B0%0A%20%20%20%20%20%20%20%20return%20%5Bt%20for%20t%20in%20cls.TITLES%20if%20t.endswith%28endswith%29%5D%0A%0A%0Aprint%28Title.allowed_titles_starting_with%28%22M%22%29%29%0Aprint%28Title.allowed_titles_ending_with%28%22s%22%29%29&cumulative=false&curInstr=0&heapPrimitives=nevernest&mode=display&origin=opt-frontend.js&py=3&rawInputLstJSON=%5B%5D&textReferences=false)에서
실행하면서 확인할 수 있다.

둘째, 리턴값으로 해당 클래스의 인스턴스를 생성하는 클래스 메서드를 사용하는 경우가 종종 있다.
이렇게 하면 해당 클래스의 인스턴스를 생성하기 위한 준비사항을 이 클래스 메서드가 알아서 처리해준다. 
예를 들어, 아래 `Person` 클래스의 `from_text_file` 메서드는 
특정 텍스트 파일에 저장된 정보를 확인한 후 그 정보를 적절히 활용하여
`Person` 클래스의 인스턴스를 생성해준다. 

In [2]:
class Person:

    def __init__(self, name, surname):
        self.name = name
        self.surname = surname

    @classmethod
    def fromDict(cls, nameDict):
        # {'이름': 'Jane', '성': 'Doe'} 형식의 사전 자료형에서
        # 이름과 성 정보를 추출해서 Person 클래스 객체 생성
        params = nameDict.values()
        return cls(*params)
    
janeDoe = {'이름':'Jane', '성':'Doe'}

jDoe = Person.fromDict(janeDoe)

print(jDoe.name)
print(jDoe.surname)

Jane
Doe


#### PythonTutor 활용 2

위 코드를 
[PythonTutor: 클래스 장식자 예제 2](http://pythontutor.com/visualize.html#code=class%20Person%3A%0A%0A%20%20%20%20def%20__init__%28self,%20name,%20surname%29%3A%0A%20%20%20%20%20%20%20%20self.name%20%3D%20name%0A%20%20%20%20%20%20%20%20self.surname%20%3D%20surname%0A%0A%20%20%20%20%40classmethod%0A%20%20%20%20def%20fromDict%28cls,%20nameDict%29%3A%0A%20%20%20%20%20%20%20%20%23%20%7B'%EC%9D%B4%EB%A6%84'%3A%20'Jane',%20'%EC%84%B1'%3A%20'Doe'%7D%20%ED%98%95%EC%8B%9D%EC%9D%98%20%EC%82%AC%EC%A0%84%20%EC%9E%90%EB%A3%8C%ED%98%95%EC%97%90%EC%84%9C%0A%20%20%20%20%20%20%20%20%23%20%EC%9D%B4%EB%A6%84%EA%B3%BC%20%EC%84%B1%20%EC%A0%95%EB%B3%B4%EB%A5%BC%20%EC%B6%94%EC%B6%9C%ED%95%B4%EC%84%9C%20Person%20%ED%81%B4%EB%9E%98%EC%8A%A4%20%EA%B0%9D%EC%B2%B4%20%EC%83%9D%EC%84%B1%0A%20%20%20%20%20%20%20%20params%20%3D%20nameDict.values%28%29%0A%20%20%20%20%20%20%20%20return%20cls%28*params%29%0A%20%20%20%20%0AjaneDoe%20%3D%20%7B'%EC%9D%B4%EB%A6%84'%3A'Jane',%20'%EC%84%B1'%3A'Doe'%7D%0A%0AjDoe%20%3D%20Person.fromDict%28janeDoe%29%0A%0Aprint%28jDoe.name%29%0Aprint%28jDoe.surname%29&cumulative=false&curInstr=0&heapPrimitives=nevernest&mode=display&origin=opt-frontend.js&py=3&rawInputLstJSON=%5B%5D&textReferences=false)에서
실행하면서 확인할 수 있다.

### 정적 메서드 장식자: `@staticmethod`

정적 메서드는 클래스나 인스터를 지정하는 인자를 사용하지 않는다.
따라서 일반 함수를 선언하는 것과 완벽하게 동일하다.
다만, 클래스 내부에서 선언되었기 때문에 항상 해당 클래스의 이름을 
지정자로 사용하여 호출된다. 

또한 정적 메서드의 본문에서 해당 클래스의 인스턴스 속성과 인스턴스 메서드는
전혀 활용되지 못한다.
해당 클래스의 클래스 메서드는 클래스의 이름과 함께 사용해야 한다.
반면에 인스턴스 메서드는 클래스 메서드와 정적 메서드를 모두 활용할 수 있다. 

결론적으로, 클래스 메서드의 선언 및 활용 방식과 거의 동일하다. 
다만, 정적 메서드는 클래스를 지정할 때 `cls` 대신에 해당 클래스의 이름을 직접
언급해야 한다는 차이점이 있을 뿐이다.

아래 예제가 세 종류의 메서드 활용법을 잘 보여준다. 

In [3]:
class Person:
    TITLES = ('Dr', 'Mr', 'Mrs', 'Ms')

    def __init__(self, name, surname):
        self.name = name
        self.surname = surname

    # 인스턴스 메서드
    def fullname(self):
        return f"{self.name} {self.surname}"

    # 클래스 메서드
    @classmethod
    def allowed_titles_starting_with(cls, startswith):
        return [t for t in cls.TITLES if t.startswith(startswith)]

    # 정적 메서드
    @staticmethod
    def allowed_titles_ending_with(endswith):
        return [t for t in Person.TITLES if t.endswith(endswith)]


jane = Person("Jane", "Smith")

print(jane.fullname())

print(jane.allowed_titles_starting_with("M"))
print(Person.allowed_titles_starting_with("M"))

print(jane.allowed_titles_ending_with("s"))
print(Person.allowed_titles_ending_with("s"))

Jane Smith
['Mr', 'Mrs', 'Ms']
['Mr', 'Mrs', 'Ms']
['Mrs', 'Ms']
['Mrs', 'Ms']


#### PythonTutor 활용 3

위 코드를 
[PythonTutor: 장식자 예제](http://pythontutor.com/visualize.html#code=class%20Person%3A%0A%20%20%20%20TITLES%20%3D%20%28'Dr',%20'Mr',%20'Mrs',%20'Ms'%29%0A%0A%20%20%20%20def%20__init__%28self,%20name,%20surname%29%3A%0A%20%20%20%20%20%20%20%20self.name%20%3D%20name%0A%20%20%20%20%20%20%20%20self.surname%20%3D%20surname%0A%0A%20%20%20%20%23%20%EC%9D%B8%EC%8A%A4%ED%84%B4%EC%8A%A4%20%EB%A9%94%EC%84%9C%EB%93%9C%0A%20%20%20%20def%20fullname%28self%29%3A%0A%20%20%20%20%20%20%20%20return%20f%22%7Bself.name%7D%20%7Bself.surname%7D%22%0A%0A%20%20%20%20%23%20%ED%81%B4%EB%9E%98%EC%8A%A4%20%EB%A9%94%EC%84%9C%EB%93%9C%0A%20%20%20%20%40classmethod%0A%20%20%20%20def%20allowed_titles_starting_with%28cls,%20startswith%29%3A%0A%20%20%20%20%20%20%20%20return%20%5Bt%20for%20t%20in%20cls.TITLES%20if%20t.startswith%28startswith%29%5D%0A%0A%20%20%20%20%23%20%EC%A0%95%EC%A0%81%20%EB%A9%94%EC%84%9C%EB%93%9C%0A%20%20%20%20%40staticmethod%0A%20%20%20%20def%20allowed_titles_ending_with%28endswith%29%3A%0A%20%20%20%20%20%20%20%20return%20%5Bt%20for%20t%20in%20Person.TITLES%20if%20t.endswith%28endswith%29%5D%0A%0A%0Ajane%20%3D%20Person%28%22Jane%22,%20%22Smith%22%29%0A%0Aprint%28jane.fullname%28%29%29%0A%0Aprint%28jane.allowed_titles_starting_with%28%22M%22%29%29%0Aprint%28Person.allowed_titles_starting_with%28%22M%22%29%29%0A%0Aprint%28jane.allowed_titles_ending_with%28%22s%22%29%29%0Aprint%28Person.allowed_titles_ending_with%28%22s%22%29%29&cumulative=false&curInstr=0&heapPrimitives=nevernest&mode=display&origin=opt-frontend.js&py=3&rawInputLstJSON=%5B%5D&textReferences=false)에서
실행하면서 확인할 수 있다.

### 연습 4

1. 다음 속성과 메서드를 포함하는 클래스 `Numbers`를 정의하라.  
    * `MULTIPLIER`: 클래스 속성
    * `__init__` 메서드: 숫자 두 개를 입력받아 각각 인스턴스 속성 `x`와 `y`로 저장.
    * `add`: 인스턴스 메서드. `x`와 `y`의 합 내주기
    * `multiply`: 클래스 메서드. 하나의 숫자 `a`를 입력 받아 `MULTIPLIER`와 곱센 결과 내주기
    * `subtract`: 정적 메서드. `b`와 `c` 숫자 두 개를 입력 받아 `b-c` 내주기

In [4]:
class Numbers:
    MULTIPLIER = 3.5

    def __init__(self, x, y):
        self.x = x
        self.y = y

    def add(self):
        return self.x + self.y

    @classmethod
    def multiply(cls, a):
        return cls.MULTIPLIER * a

    @staticmethod
    def subtract(b, c):
        return b - c

In [5]:
twoAndfive = Numbers(2,5)

print(twoAndfive.add())

print(Numbers.multiply(4))
print(twoAndfive.multiply(4))

print(Numbers.subtract(7, 2))
print(twoAndfive.subtract(7,2))

7
14.0
14.0
5
5


## `object` 클래스와 매직 메서드

생성된 객체와 관련된 속성과 메서드를 확인하려면 `dir` 함수를 활용한다.
예를 들어, 다음 `Person` 클래스의 인스턴스 `jane`을 생성해보자. 

In [6]:
class Person:
    def __init__(self, name, surname):
        self.name = name
        self.surname = surname

    def fullname(self):
        return f"{self.name} {self.surname}"

jane = Person("Jane", "Smith")

이제 `dir` 함수를 이용하여 `jane`이 가리키는 객체의 속성과 메서드를 확인해보자. 

In [7]:
dir(jane)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'fullname',
 'name',
 'surname']

### 최상위 클래스: `object`

놀랍게도 `Person` 클래스를 선언할 때 명시된 속성과 메서드 이외에 
추가로 많은 이름이 보인다. 
이유는 다음과 같다.

* 파이썬의 모든 클래스는 `object`라는 클래스를 상속한다.
* 하나의 클래스를 상속하면 해당 클래스의 속성과 메서드도 모두 함께 상속받는다.
* `object` 클래스에는 위에서 언급된, 두 개의 밑줄로 감싸인 속성과 메서드가 선언되어 있다.

상속의 대상인 클래스를 **상위 클래스**(superclass) 또는 **부모 클래스**(parent class), 
상속하는 클래스를 **하위 클래스**(subclass) 또는 **자식 클래스**(child class)
라고 부른다. 
이런 의미에서 `object`는 최상위에 위치한 클래스이다.
`object` 클래스에 포함된 속성과 메서드는 모두 양끝이 이중 밑줄로 감싸이며,
`object` 클래스의 메서드를 특별히 **매직 메서드**(magic method)라 부른다. 
따라서 임의의 클래스는 `object` 클래스에서 선언된 매직 메서드와 속성을
모두 상속받는다.

`Person` 클래스의 엄밀한 정의는 다음과 같이, `object` 클래스를 상속하는 것을 
함께 명시해야 한다. 
하지만 상속 대상이 `object` 클래스 뿐인 경우 명시하지 않아도 되며,
괄호도 생략한다.

---

```python
class Person(object):
    def __init__(self, name, surname):
        self.name = name
        self.surname = surname

    def fullname(self):
        return f"{self.name} {self.surname}"
```

---


**주의:** 파이썬 2에서는 `object`를 반드시 명시해야 한다. 
상속에 대해서는 일단 이 정도만 알고 있으면 되며, 보다 자세한 설명은
추후 다룬다.

### `object` 클래스의 속성과 매직 메서드 기능

초기 설정 메서드 `__init__`는 굳이 선언될 필요가 없다고 앞서 언급하였는데,
그 이유가 바로 상속하는 부모 클래스의 `__init__` 메서드가 자동으로 사용되기 때문이다.
하지만 위와 같이 `__init__` 메서드를 선언하면 새로 정의된 함수가 사용된다.

모든 매직 메서드는 고유의 기능을 수행한다.
따라서, 특별한 사유가 없으면 매직 메서드를 다시 정의하는 일은 피해야 한다.
여기서는 주요 매직 메서드의 기본 기능을 간략하게 확인하고 자세한 설명은 하지 않는다.

#### `__repr__` 메서드와 `__str__` 메서드

숫자, 문자열, 리스트, 튜플, 사전 등을 확인하거나 출력하면
우리에게 매우 친숙한 방식으로 보여진다. 
예를 들어, 리스트의 경우는 값을 확인하는 거와 출력하는 데에 차이가 없다.

In [8]:
[1, 2, 3]

[1, 2, 3]

In [9]:
print([1, 2, 3])

[1, 2, 3]


반면에 문자열의 경우는 조금 다르다.

In [10]:
"파이썬이 최고에요!"

'파이썬이 최고에요!'

In [11]:
print("파이썬이 최고에요!")

파이썬이 최고에요!


이렇게 작동하는 이유는 문자열 클래스의 내부에서
`__repr__` 과 `__str__` 두 메서드가 조금 다르게 정의되어 있기 때문이다.

값을 확인할 때는 `__repr__` 메서드가 호출되고,
출력할 때는 `__str__` 메서드가 호출된다.

In [12]:
"파이썬이 최고에요!".__repr__()

"'파이썬이 최고에요!'"

작은 인용부호가 포함된 문자열이 리턴값이다.
반면에 `__str__` 메서드의 경우 작은 인용부호는 포함되지 않은 문자열이 사용되었다. 

In [13]:
"파이썬이 최고에요!".__str__()

'파이썬이 최고에요!'

이제 `jane` 객체를 확인하고 출력해보자.

In [14]:
jane

In [15]:
print(jane)

두 경우 아주 조금 다르기는 기본적으로 동일한 정보를 보여준다. 
보여지는 정보는 `jane`이 `Person` 클래스의 객체를 가리킨다는 사실과
해당 객체가 저장되어 있는 메모리의 주소이다. 
이렇게 나오는 이유는 `Person` 클래스에서 `__repr__`, `__str__` 모두
정의되어 있지 않기 때문이다. 

이제 `Person` 클래스에서 `__repr__`, `__str__` 두 메서드를 다음과 같이 재정의해보자. 
두 메서드 모두 리턴값은 문자열이어야 한다. 

In [16]:
class Person(object):
    def __init__(self, name, surname):
        self.name = name
        self.surname = surname

    def fullname(self):
        return f"{self.name} {self.surname}"
    
    def __str__(self):
        return f"성: {self.surname}, 이름: {self.name}"

    def __repr__(self):
        return f"Person(성: {self.surname}, 이름: {self.name})"

jane = Person("Jane", "Smith")

다시 `jane`을 확인하고 출력해보자. 

In [17]:
# __repr__ 메서드 사용됨.
jane

Person(성: Smith, 이름: Jane)

In [18]:
# __str__ 메서드 사용됨.
print(jane)

성: Smith, 이름: Jane


그리고 `__str__` 메서드가 재정의되어 있지 않은 경우,
`__repr__` 메서드의 정의를 사용한다.

In [19]:
class Person(object):
    def __init__(self, name, surname):
        self.name = name
        self.surname = surname

    def fullname(self):
        return f"{self.name} {self.surname}"
    
    def __repr__(self):
        return f"Person(성: {self.surname}, 이름: {self.name})"

jane = Person("Jane", "Smith")

In [20]:
jane

Person(성: Smith, 이름: Jane)

In [21]:
print(jane)

Person(성: Smith, 이름: Jane)


##### 차이점

`__repr__` 메서드와 `__str__` 메서드는 기본적으로 비슷한 용도로 사용된다.
차이점은 `__str__` 메서드는 객체들을 적절하게 표현하는 데에 사용되며,
`__repr__` 메서드는 객체들을 좀 더 형식을 갖추어 표현할 때 사용한다.

예를 들어, 시간과 관련된 `datetime` 모듈의 `date` 클래스와
`datetime` 클래스 객체를 이용하여 두 함수가 다르게 작동하는 것을 살펴보자.

아래 예제에서는 `repr`와 `str` 두 함수를 소개하면서 두 메서드의 
차이점을 설명한다. 
`repr` 함수를 호출하면 인자로 사용된 값의 `__repr__` 메서드가,
반면에 `str` 함수를 호출하면 인자로 사용된 값의 `__str__` 메서드가 호출된다.

In [22]:
import datetime

# datetime 객체: 현재 시간 정보 저장
now = datetime.datetime.now()

# date 객체: 현재 날짜 정보 저장
today = datetime.date.today()

`repr` 함수는 훨씬 형식을 갖춘 표현을 사용한다.

In [23]:
repr(now)

'datetime.datetime(2020, 5, 25, 2, 26, 54, 54973)'

In [24]:
repr(today)

'datetime.date(2020, 5, 25)'

반면에 `str` 함수는 간소화된 표현을 사용한다.

In [25]:
str(now)

'2020-05-25 02:26:54.054973'

In [26]:
str(today)

'2020-05-25'

#### `__class__` 속성

객체의 자료형을 확인할 때 `type` 함수를 사용한다.
그러면 `type` 함수는 해당 객체의 `__class__` 속성을 확인한다.

In [27]:
jane.__class__

__main__.Person

In [28]:
type(jane)

__main__.Person

In [29]:
[1, 2, 3].__class__

list

In [30]:
type([1, 2, 3])

list

#### `__dict__` 속성

`__dict__` 속성은 인스턴스 속성들을 사전으로 모아 둔다.

In [31]:
jane.__dict__

{'name': 'Jane', 'surname': 'Smith'}

즉, `__dict__`는 해당 인스턴스의 모든 속성을 사전 자료형으로 포함한다. 
키는 속성 변수, 값은 속성 값을 사용한다. 

반면에 리스트는 인스턴스 속성을 전혀 갖지 않는다.

In [32]:
[1, 2, 3].__dict__

AttributeError: 'list' object has no attribute '__dict__'

#### `__iter__` 메서드, 이터러블 자료형, 이터레이터 자료형

리스트, 튜플, 문자열 등은 **이터러블**(iterable) 자료형에 속한다. 
이터러블 자료형은 `for` 반복문과 함께 사용될 수 있는 자료형이라고 생각하면 쉽게 
이해할 수 있다. 
보다 엄밀히 말하면 `__iter__` 메서드가 정의되어 있는 자료형이 이터러블 자료형이다.
실제로 모든 리스트, 튜플, 문자열은 해당 메서드를 포함하고 있다.

In [33]:
[1, 2, 3].__iter__()

In [34]:
(1, 2, 3).__iter__()

In [35]:
'1, 2, 3'.__iter__()

`__iter__` 메서드의 역할은 해당 자료형을 **이터레이터**(iterator)로
변환시키는 일이다. 
이터레이터는 `__next__` 메서드를 포함하는 자료형이다.
해당 메서드는, 예를 들어, `for` 반목문에서 각 항목을 활용할 때 사용된다. 

리스트 `[1, 2, 3]`을 이용하는 예를 살펴보자.

In [36]:
aList = [1, 2, 3]

`iter` 함수를 사용하면 `__iter__` 메서드가 사용된다.

In [37]:
aIterator = iter(aList)

이제 `next` 함수를 호출하면 `__next__` 메서드가 호출된다.

In [38]:
next(aIterator)

1

`next` 함수를 호출할 때마다 리스트의 다음 항목이 확인된다.

In [39]:
next(aIterator)

2

In [40]:
next(aIterator)

3

마지막 항목이 확인된 후에는 `StopIteration` 오류가 발생한다. 

In [41]:
next(aIterator)

StopIteration: 

`next` 함수를 다시 사용하려면 이터레이터를 다시 생성해야 한다. 
이번에는 `__iter__`와 `__next__` 메서드를 직접 사용한다.

In [42]:
aIterator = aList.__iter__()

In [43]:
aIterator.__next__()

1

In [44]:
aIterator.__next__()

2

In [45]:
aIterator.__next__()

3

In [46]:
aIterator.__next__()

StopIteration: 

실제로 이터러블 자료형을 `for` 반복문과 함께 사용하면
내부적으로는 앞서 설명한 대로 이터레이터로 변환한 후 
`next` 함수를 반복적으로 사용한다. 

예를 들어, 아래 `for` 반복문을 살펴보자.

In [47]:
for item in [1, 2, 3]:
    print(item)

1
2
3


위 반복문을 파이썬 해석기가 처리하는 실제 내용은 다음과 같다. 

In [48]:
aIterator = iter([1, 2, 3])

# 무한 반복문
while True:
    try:
        # 다음 항목 구하기
        item = next(aIterator)
        print(item)
    except StopIteration:
        # 더 이상 확인할 항목이 없으면 반복문 종료
        break

1
2
3


이터러블 자료형과 이터레이터 자료형에 대해서는 우선 이 정도 알아두면 좋다. 
보다 많은 활용 예제에 대한 설명은 
[programiz.com: Python Iterators](https://www.programiz.com/python-programming/iterator)을 
참고할 것을 추천한다.

#### `__len__` 메서드, 순차 자료형, 모음 자료형

리스트, 튜플, 문자열, `range` 등의 자료형처럼 항목들의 순서와 중복이 허용되는 
자료형을 **순차** 또는 **시퀀스**(sequence) 자료형이라 한다. 
반면에 집합, 사전 등의 자료형처럼 순서를 무시하고 중복이 허용되지 않는 자료형은
단순히 **모음**(collection) 자료형이라 부른다. 
경우에 따라 순차 자료형도 단순히 모음 자료형으로 간주하기도 한다. 

순차와 모음 자료형은 모두 항목의 개수를 확인해주는 `__len__` 메서드를 갖고 있다.

In [49]:
[1, 2, 3].__len__()

3

In [50]:
"Python".__len__()

6

`jane`의 `__dict__` 속성에는 두 개의 항목이 포함되어 있다.

In [51]:
jane.__dict__.__len__()

2

이와 같이 `__len__` 메소드를 포함한 자료형에 대해서
`len` 함수를 사용하여 항목의 개수를 확인할 수 있다. 

In [52]:
len([1, 2, 3])

3

In [53]:
len("Python")

6

In [54]:
len(jane.__dict__)

2

`len` 함수를 아래와 같이 정의된 것으로 생각할 수 있다. 

```python
def len(s):
    return s.__len__()
```

## 매직 메서드 재정의

`Person` 클래스에서 `__init__` 메서드를 새로 정의하였다.
이와 같이 메서드를 새로 정의하는 것을 
**메서드 재정의** 또는 **메서드 오버라이딩**(method overriding)이라 부른다. 
따라서 모든 매직 메서드는 재정의 되든가, 아니면 부모 클래스에서 선언된 그대로 사용된다. 
여기서는 앞서 다루지 않았지만 중요한 매직 메서드 몇 개를 재정의하면서
유용한 자료형을 정의하는 방법을 소개한다. 

### `Fractional` 클래스

부동소수점을 다루는 `float` 자료형에 속하는 값들은 실수를 다루기 위해 구현되었다.
하지만 컴퓨터의 함계로 인해 실제로 다룰 수 있는 값들의 소수점 이하의 자릿수가 제한되며,
따라서 컴퓨터는 기본적으로 유리수만 다룰 수 있다. 

유리수의 또다른 정의는 정수들의 분수로 표현될 수 있는 실수를 말한다. 
하지만 정수들의 분수를 실제로 부동소수점으로 처리하면 또한 컴퓨터의 한계로 인해
계산이 부정확한 경우가 종종 발생한다. 
따라서 정수들의 분수를 독립된 자료형으로 다룰 필요가 발생한다.
이를 위해 파이썬은 `Fractional` 이란 자료형을 제공한다. 

`Fractional` 클래스는 `fractions` 모듈에 선언되어 있다. 
따라서 정수들의 분수를 사용하려면 먼저 해당 클래스를 불러와야 한다. 

In [55]:
from fractions import Fraction

2/3에 해당하는 수는 아래와 같이 표현한다.

**주의:** 예제에서 사용되는 변수의 접두어로 사용되는 `R`은 유리수(rational)을 가리킨다.
유리수임을 강조하기 위해 일부러 사용할 뿐이지 관례는 아니다.

In [56]:
R12= Fraction(1, 2)
R13= Fraction(1, 3)
R24= Fraction(2, 4)

사칙연산이 가능하다.

In [57]:
# 1/2 + 1/3 = 5/6

R12 + R13

Fraction(5, 6)

In [58]:
# 1/2 * 1/3 = 1/6

R12 * R13

Fraction(1, 6)

In [59]:
# (1/3) * (1/2) = 2/3

R13 / R12

Fraction(2, 3)

`print` 함수를 사용하면 익숙한 표현으로 보여진다. 
즉, `__str__` 메소드가 적절하게 정의되어 있다.

In [60]:
print(R12)

1/2


In [61]:
print(R13)

1/3


In [62]:
print(R13 / R12)

2/3


### 연습: 유리수 클래스 선언하기

`Fractional` 클래스와 유사하게 작동하는 클래스를 직접 정의하면서
다음 매직 메서드들을 재정의하는 실습을 해보자.
재정의 대상 메서드는 다음과 같다.

* `__repr__`
* `__str__`
* `__eq__`
* `__lt__`
* `__le__`
* `__add__`

In [63]:
import math

class myFraction:
    """분수 클래스

    정수들의 분수를 정수들의 쌍으로 구현함.
    정부들의 분수는 기본적으로 약분 처리하여 저장.
    n
    
    """

    def __init__(self, numerator, denominator, _normalize=True):
        
        if denominator == 0:
            raise ZeroDivisionError(f"myFraction({numerator}, 0)")
        elif denominator < 0:
            denominator = -denominator
            numerator = -numerator
        
        if _normalize:
            g = math.gcd(numerator, denominator)
            if denominator < 0:
                g = -g
        
            numerator //= g
            denominator //= g
        
        self._numerator = numerator
        self._denominator = denominator
                
    def __repr__(self):
        return f"{self.__class__.__name__}({self._numerator},{self._denominator})"
    
    def __str__(self):
        if self._denominator == 1:
            return str(self._numerator)
        else:
            return f"{self._numerator}/{self._denominator}"
    
    def __eq__(self, other):
        return (self._numerator * other._denominator) \
                == (other._numerator * self._denominator)

    def __lt__(self, other):
        return (self._numerator * other._denominator) \
                < (other._numerator * self._denominator)

    def __le__(self, other):
        return self < other or self == other

    def __add__(self, other):
        numerator = self._numerator * other._denominator + \
                    self._denominator * other._numerator
        denominator = self._denominator * other._denominator
        return myFraction(numerator, denominator)

In [64]:
R14 = myFraction(1, 4)

In [65]:
Rm14 = myFraction(1, -4)

In [66]:
R34 = myFraction(3, 4)

In [67]:
Rm34 = myFraction(-3, 4)

In [68]:
R28 = myFraction(2, 8)

In [69]:
print(R14)

1/4


In [70]:
print(R28)

1/4


In [71]:
R14 == R28

True

In [72]:
R14 < R34

True

In [73]:
Rm34 < Rm14

True

In [74]:
Rm34 <= Rm14

True

In [75]:
Rm34 >= Rm14

False

In [76]:
print(R14 + R34)

1


In [77]:
print(Rm14 + R34)

1/2


#### PythonTutor 활용 2

위 예제들을 
[PythonTutor: 유리스 클래스]()에서
실행하면서 확인할 수 있다.